In [ ]:
import orchid

In [ ]:
# Remember that importing `orchid` results in a call to `pythonnet.load()`
# to specify the targeted runtime:
#
# 'netfx': .NET Framework
# 'coreclr': .NET Core
# 'mono': Mono

import clr

In [ ]:
from System import (
    ArgumentException,
    DateTime,
    DateTimeKind,
    DateTimeOffset,
    Int32,
    TimeSpan,
)

In [ ]:
DateTime

In [ ]:
DateTime.Overloads, DateTime.__overloads__

In [ ]:
type(DateTime.__overloads__)

In [ ]:
dir(DateTime)

In [ ]:
import pythonnet

In [ ]:
pythonnet.get_runtime_info()

In [ ]:
type(DateTimeKind.Utc)

In [ ]:
dir(DateTimeKind.Utc)

In [ ]:
DateTimeKind.Utc.GetType()

In [ ]:
dir(DateTimeKind.Utc.GetType())

In [ ]:
DateTimeKind.Utc.GetType().BaseType

In [ ]:
DateTimeKind.Utc.GetType().BaseType.FullName

In [ ]:
[n for n in dir(DateTimeOffset) if 'Equality' in n]

In [ ]:
DateTimeOffset.op_Equality.__overloads__

In [ ]:
try:
    print('Trying `108 == DateTimeOffset.Value`')
    108 == DateTimeOffset.MaxValue
except TypeError as te:
    print(f'TypeError: {te}')

In [ ]:
try:
    print('Trying `108 == TimeSpan.MaxValue`')
    108 == TimeSpan.MaxValue
except TypeError as te:
    print(f'TypeError: {te}')

We filed an issue with the Python.NET team. The fairly quickly responded with the following:

"""
Yes, we tried to limit the "implicit" conversions to a minimum. I don't even know, which
change in particular is responsible for the behavioural change fix that you are observing
here, but you are only able to compare things to a .NET object that are directly convertible
to it. If you'd really require this for DateTimeOffset and TimeSpan, you could make them
convertible via a Codec. Otherwise, I'd suggest you just generate the respective comparison
values using .FromTicks.
"""

In [ ]:
108 == DateTimeOffset.MaxValue.Ticks

In [ ]:
108 == TimeSpan.MaxValue.Ticks

In [ ]:
108 == TimeSpan.MinValue.Ticks

Python.NET 2.5.2 allowed expressions like `TimeSpan()`. (Note that the .NET
`TimeSpan` class **does not** have a default constructor.) This expression
is no longer supported. Instead, one must supply an argument (typically
zero (0)) to the constructor or to methods like `TimeSpan.FromTicks()`.

In [ ]:
TimeSpan(8801)

In [ ]:
try:
    print('Trying expression, `TimeSpan()`')
    TimeSpan()
except TypeError as te:
    print(f'TypeError: {te}')

In [ ]:
TimeSpan(0)

In [ ]:
TimeSpan.FromTicks(0)

During integration testing, we discovered an issue setting an attribute with
type, `Int32`, using a Python `int` of value 7. The run-time reported that
the types, `Int32` and `PyInt` were incompatible. This scenario requires
significant set up.

In [ ]:
# Find the well named 'Demo_1H'
bakken = orchid.load_project('c:/src/Orchid.IntegrationTestData/frankNstein_Bakken_UTM13_FEET.ifrac')
candidate_wells = list(bakken.wells().find_by_name('Demo_1H'))
assert len(candidate_wells) == 1
demo_1h = candidate_wells[0]

In [ ]:
# Create an attribute with name, 'My New Attribute', and type, `System.Int32`
from Orchid.FractureDiagnostics.Factories.Implementations import Attribute

attribute_to_add_type = Int32
attribute_to_add = Attribute[attribute_to_add_type].Create('My New Attribute')

In [ ]:
# Add newly created attribute to well, 'Demo_1H'
with orchid.dot_net_disposable.disposable(demo_1h.dom_object.ToMutable()) as mutable_well:
    mutable_well.AddStageAttribute(attribute_to_add)

In [ ]:
# Find stage number 7 in well, 'Demo_1H'
maybe_stage = demo_1h.stages().find_by_display_stage_number(7)
assert maybe_stage is not None
stage_7 = maybe_stage

In [ ]:
# Add attribute with value, 17, to stage 7, with Python `int` type.
with (orchid.dot_net_disposable.disposable(stage_7.dom_object.ToMutable())) as mutable_stage:
    # This action will fail because the attribute type is `System.Int32`
    # and `pythonnet-3.0.0.post1` **does not** implicitly equate these two types.
    try:
        mutable_stage.SetAttribute(attribute_to_add, int)
    except ArgumentException as ae:
        print(f'ArgumentException: {ae}')


In [ ]:
# Add attribute with value, 17, to stage 7
with (orchid.dot_net_disposable.disposable(stage_7.dom_object.ToMutable())) as mutable_stage:
    mutable_stage.SetAttribute(attribute_to_add, attribute_to_add_type(7))


In [ ]:
# Verify added attribute value
ignored_object = object()
is_attribute_present, actual_attribute_value = stage_7.dom_object.TryGetAttributeValue(attribute_to_add,
                                                                                       ignored_object)
assert is_attribute_present
assert type(actual_attribute_value) == int
assert actual_attribute_value == 7